In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [7]:
train = pd.read_csv("../train.csv")
test = pd.read_csv("../test.csv")
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856387 entries, 0 to 856386
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   IntersectionId           856387 non-null  int64  
 1   Latitude                 856387 non-null  float64
 2   Longitude                856387 non-null  float64
 3   EntryStreetName          848239 non-null  object 
 4   ExitStreetName           850100 non-null  object 
 5   EntryHeading             856387 non-null  object 
 6   ExitHeading              856387 non-null  object 
 7   Hour                     856387 non-null  int64  
 8   Weekend                  856387 non-null  int64  
 9   Month                    856387 non-null  int64  
 10  Path                     856387 non-null  object 
 11  TotalTimeStopped_p20     856387 non-null  float64
 12  TotalTimeStopped_p40     856387 non-null  float64
 13  TotalTimeStopped_p50     856387 non-null  float64
 14  Tota

## One-hot encoding

In [9]:
# StreetName, Path 삭제
str_col_onehot = ['City', 'EntryHeading', 'ExitHeading']

train_onehot = train.copy()
test_onehot = test.copy()

In [10]:
X = pd.get_dummies(train_onehot, columns=str_col_onehot)
Y = train[['TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80', 'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80']]
for col in X.columns:
    if X[col].dtype == 'object':
        X.drop(col, axis = 1, inplace = True)
    if col.find('Stop') != -1:
        X.drop(col, axis = 1, inplace = True)

In [11]:
X

,IntersectionId,Latitude,Longitude,Hour,Weekend,Month,City_Atlanta,City_Boston,City_Chicago,City_Philadelphia,...,EntryHeading_SW,EntryHeading_W,ExitHeading_E,ExitHeading_N,ExitHeading_NE,ExitHeading_NW,ExitHeading_S,ExitHeading_SE,ExitHeading_SW,ExitHeading_W
0,0,33.791659,-84.430032,0,0,6,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,33.791659,-84.430032,0,0,6,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,33.791659,-84.430032,1,0,6,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,33.791659,-84.430032,1,0,6,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,33.791659,-84.430032,2,0,6,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856382,1990,39.938232,-75.149930,17,1,12,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
856383,1990,39.938232,-75.149930,18,1,12,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
856384,1990,39.938232,-75.149930,19,1,12,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
856385,1990,39.938232,-75.149930,20,1,12,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856387 entries, 0 to 856386
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   IntersectionId     856387 non-null  int64  
 1   Latitude           856387 non-null  float64
 2   Longitude          856387 non-null  float64
 3   Hour               856387 non-null  int64  
 4   Weekend            856387 non-null  int64  
 5   Month              856387 non-null  int64  
 6   City_Atlanta       856387 non-null  uint8  
 7   City_Boston        856387 non-null  uint8  
 8   City_Chicago       856387 non-null  uint8  
 9   City_Philadelphia  856387 non-null  uint8  
 10  EntryHeading_E     856387 non-null  uint8  
 11  EntryHeading_N     856387 non-null  uint8  
 12  EntryHeading_NE    856387 non-null  uint8  
 13  EntryHeading_NW    856387 non-null  uint8  
 14  EntryHeading_S     856387 non-null  uint8  
 15  EntryHeading_SE    856387 non-null  uint8  
 16  En

In [13]:
X.columns

Index(['IntersectionId', 'Latitude', 'Longitude', 'Hour', 'Weekend', 'Month',
       'City_Atlanta', 'City_Boston', 'City_Chicago', 'City_Philadelphia',
       'EntryHeading_E', 'EntryHeading_N', 'EntryHeading_NE',
       'EntryHeading_NW', 'EntryHeading_S', 'EntryHeading_SE',
       'EntryHeading_SW', 'EntryHeading_W', 'ExitHeading_E', 'ExitHeading_N',
       'ExitHeading_NE', 'ExitHeading_NW', 'ExitHeading_S', 'ExitHeading_SE',
       'ExitHeading_SW', 'ExitHeading_W'],
      dtype='object')

In [14]:
ml = RandomForestRegressor()
ml.fit(X, Y)

RandomForestRegressor()

In [15]:
predict_X = pd.get_dummies(test_onehot, columns = str_col_onehot)

for col in predict_X.columns:
    if predict_X[col].dtype == 'object':
        predict_X.drop(col, axis = 1, inplace = True)
    if col.find('Stop') != -1:
        predict_X.drop(col, axis = 1, inplace = True)

In [16]:
predict_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1921357 entries, 0 to 1921356
Data columns (total 26 columns):
 #   Column             Dtype  
---  ------             -----  
 0   IntersectionId     int64  
 1   Latitude           float64
 2   Longitude          float64
 3   Hour               int64  
 4   Weekend            int64  
 5   Month              int64  
 6   City_Atlanta       uint8  
 7   City_Boston        uint8  
 8   City_Chicago       uint8  
 9   City_Philadelphia  uint8  
 10  EntryHeading_E     uint8  
 11  EntryHeading_N     uint8  
 12  EntryHeading_NE    uint8  
 13  EntryHeading_NW    uint8  
 14  EntryHeading_S     uint8  
 15  EntryHeading_SE    uint8  
 16  EntryHeading_SW    uint8  
 17  EntryHeading_W     uint8  
 18  ExitHeading_E      uint8  
 19  ExitHeading_N      uint8  
 20  ExitHeading_NE     uint8  
 21  ExitHeading_NW     uint8  
 22  ExitHeading_S      uint8  
 23  ExitHeading_SE     uint8  
 24  ExitHeading_SW     uint8  
 25  ExitHeading_W     

In [17]:
predict_Y = ml.predict(predict_X)
predict_Y

array([[ 0.16 ,  2.05 , 22.05 ,  1.062, 10.587, 56.045],
       [ 0.   ,  0.19 , 18.33 ,  0.   ,  0.649, 42.57 ],
       [ 0.   ,  0.19 ,  2.46 ,  0.   ,  0.649, 13.088],
       ...,
       [ 0.   ,  0.   ,  4.01 ,  0.   ,  0.   , 21.422],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [18]:
predict_Y.size

11528142

In [19]:
# predict_Y = pd.DataFrame(predict_Y)
# predict_Y.rename(columns= {0: '_0', 1: '_1', 2: '_2', 3: '_3', 4: '_4', 5: '_5'}, inplace = True)
# a = predict_Y.head().groupby(level=0)
# a.groups
predict_Y = pd.DataFrame(np.ravel(predict_Y))
predict_Y_index = (predict_Y.index // 6).astype('string') + '_' + (predict_Y.index % 6).astype('string')
predict_Y.set_index(predict_Y_index, inplace=True)

In [20]:
submission = predict_Y.loc[:'1920334_5']

In [21]:
submission.to_csv("submission_rf.csv")